In [437]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

season = '8'

url = 'https://en.wikipedia.org/wiki/The_Great_British_Bake_Off_(series_{})'.format(season)
r = requests.get(url)

In [438]:
soup = BeautifulSoup(r.content, 'html.parser')


In [439]:
episode_tables = soup.select('h3 + p + table')
episode_tables = [str(table) for table in episode_tables]

episode_names = soup.select('p + table ~ h3')
episode_names = [name.span['id'] for name in episode_names]
print(len(episode_tables))
print(len(episode_names))

12
13


In [440]:
episode_dfs = pd.read_html(''.join(episode_tables), header=0)

for i, episode in enumerate(episode_dfs):
    try:
        challenge_list = [re.findall('\((.*)\)',challenge)[0] for challenge in episode.columns[1:]]
        episode_overview = {
            "Signature": challenge_list[0],
            "Technical": challenge_list[1],
            "Showstopper": challenge_list[2]
        }
    except:
        episode_overview = {}
    
    episode.columns = ['Baker', 'Signature', 'Technical', 'Showstopper']
    
    episode_name_split = episode_names[i].split(':')
    
    if len(episode_name_split) == 2:
        episode_number = episode_names[i].split(':')[0].split('_')[1]
        episode_theme = episode_names[i].split(':')[1]
    else:
        episode_number = 'special'
        episode_theme = episode_names[i]
        
    episode_overview['episode_number'] = episode_number
    episode_overview['theme'] = episode_theme
    episode_overview['season'] = season
    
    episode['episode_number'] = episode_number
    episode['episode_theme'] = episode_theme
    episode['season'] = season

In [441]:
episode_dfs[3]

,Baker,Signature,Technical,Showstopper,episode_number,episode_theme,season
0,James,'Windy City' Millionaire Shortbreads,2nd,Nutty Caramel Cake,4,_Caramel,8
1,Julia,Pecan Salted Caramel Millionaire Shortbreads,9th,Poppy Seed Caramel Cake,4,_Caramel,8
2,Kate,Salted Bay Caramel Millionaire Shortbreads,6th,Sticky Toffee Apple Caramel Cake,4,_Caramel,8
3,Liam,Salted Peanut Butter Millionaire Shortbreads,3rd,White Layered Ginger Caramel Cake,4,_Caramel,8
4,Sophie,Jaffa Millionaire Shortbreads,4th,Bird's Nest Caramel Cake,4,_Caramel,8
5,Stacey,Rum Salted Caramel Millionaire Shortbreads,1st,Chocolate and Coffee Caramel Cake,4,_Caramel,8
6,Steven,Orange Macadamia Nut Millionaire Shortbreads,7th,Caramel Crown Cake,4,_Caramel,8
7,Tom,Stem Ginger Caramel Shortbreads,5th,Hummingbird Cake,4,_Caramel,8
8,Yan,'Lil' Red's Ginger and Nuts' Millionaire Short...,8th,Animals of the Rainforest Caramel Cake,4,_Caramel,8


In [442]:
season_df = pd.concat(episode_dfs)

season_df

,Baker,Signature,Technical,Showstopper,episode_number,episode_theme,season
0,Chris,Tropical Fresh Fruit Cake,12th,'Raised Pork Pie and Egg Surprise' Cake,1,_Cake,8
1,Flo,Citrus Layer Cake,9th,'One in a Melon' Cake,1,_Cake,8
2,James,"Orange, Rhubarb & Ginger Crumble Cake",2nd,'Mock-accino and Croissant' Cake,1,_Cake,8
3,Julia,Courgette and Lemon Cake,3rd,'Russian Doll' Cake,1,_Cake,8
4,Kate,Moroccan Strawberry and Rose Cake,1st,'Chocolate Terrarium' Cake,1,_Cake,8
5,Liam,Apple and Almond Crumble Maple Loaf,8th,'Stackin' Sunday' Cake,1,_Cake,8
6,Peter,Coconut Cake,10th,'Bread on a Breadboard' Cake,1,_Cake,8
7,Sophie,Pineapple and Coconut Sandwich Cake,5th,'Champagne Bottle and Bucket' Cake,1,_Cake,8
8,Stacey,Granny Smith Apple and Walnut Cake,11th,'Black Evening Clutch Bag' Cake,1,_Cake,8
9,Steven,Bonfire Night Cake,6th,'A Baker's Lunch' Cake,1,_Cake,8


In [443]:
episode_overview

{'Signature': '6 Snowy Bombe Alaska Tarts',
 'Technical': 'Pistachio Kransekake Christmas Tree',
 'Showstopper': 'Magical Ice Cake',
 'episode_number': 'special',
 'theme': 'The_Big_Festive_Bake_Off',
 'season': '8'}

In [444]:
summary_tables = soup.select('h2 + table')

bakers_table = str(summary_tables[0])

elimination_key = soup.select('p + dl > dd')

elimination_table = summary_tables[1]

In [445]:
bakers_df = pd.read_html(bakers_table, header=0)[0]

bakers_df

,Baker [6][7],Age,Occupation,Hometown,Links
0,Chris Geiger,50,Software developer,Bristol,[8]
1,Flo Atkins,71,Retired,Merseyside,[9]
2,James Hillery,46,Banker,"Brentwood, Essex",[10]
3,Julia Chernogorova,21,Aviation Broker,"Crawley, West Sussex",[11]
4,Kate Lyon,29,Health and safety inspector,Merseyside,[12]
5,Liam Charles,19,Student,North London,[13]
6,Peter Abatan,52,IT programme manager,"Southend, Essex",[14]
7,Sophie Faldo,33,Former army officer and trainee stuntwoman,"West Molesey, Surrey",[15]
8,Stacey Hart,42,Former school teacher,"Radlett, Hertfordshire",[16]
9,Steven Carter-Bailey,34,Marketer,"Watford, Hertfordshire",[17]


In [446]:
elimination_colour_key = {}

for key in elimination_key:
    colour = re.findall('background-color:(\w*);', key.span['style'])[0].lower()
    value = key.contents[1]
    elimination_colour_key[colour] = value

elimination_colour_key

{'plum': " Baker was one of the judges' least favourite bakers that week, but was not eliminated.",
 'cornflowerblue': " Baker was one of the judges' favourite bakers that week, but was not the Star Baker.",
 'lightblue': ' Baker got through to the next round.',
 'orangered': ' Baker was eliminated.',
 'lemonchiffon': ' Baker was the Star Baker.',
 'limegreen': ' Baker was a series runner-up.',
 'yellow': ' Baker was the series winner.'}

In [447]:
def parse_rounds_table(table, keys):
        
    rows = table.tbody.find_all('tr')
    
    column_names = rows[1].getText().split('\n')[1:]
    
    parsed_table = [column_names]
    
    for row in rows[2:]:
        columns = row.find_all('td')

        name = re.sub('\s+', '', columns[0].getText())
        rounds = [name]

        for col in columns[1:]:
            colour = re.sub('background:','',col['style'])[:-1].lower()
            if colour in keys.keys():
                rank = keys[colour][1:]
            else:
                rank = 'out'
            try:
                multiplier = int(col['colspan'])
            except:
                multiplier = 1
            rounds.extend(multiplier * [rank])
            
        parsed_table.append(rounds)
    
    return parsed_table

In [448]:
elimination_df = pd.DataFrame(parse_rounds_table(elimination_table, elimination_colour_key))

elimination_df.columns = elimination_df.iloc[0]

elimination_df = elimination_df.iloc[1:]

elimination_df['season'] = season

elimination_df

,Baker,1,2,3,4,5,6,7,8,9,10,,None,None,None,season
1,Sophie,Baker was one of the judges' favourite bakers ...,Baker was one of the judges' favourite bakers ...,Baker got through to the next round.,Baker got through to the next round.,Baker was the Star Baker.,Baker was one of the judges' favourite bakers ...,Baker was one of the judges' favourite bakers ...,Baker got through to the next round.,Baker was the Star Baker.,Baker was the series winner.,None,None,None,None,8
2,Kate,Baker got through to the next round.,Baker was one of the judges' least favourite b...,Baker got through to the next round.,Baker was the Star Baker.,Baker got through to the next round.,Baker got through to the next round.,Baker was one of the judges' least favourite b...,Baker was one of the judges' least favourite b...,Baker was one of the judges' favourite bakers ...,Baker was a series runner-up.,None,None,None,None,8
3,Steven,Baker was the Star Baker.,Baker was the Star Baker.,Baker was one of the judges' favourite bakers ...,Baker got through to the next round.,Baker got through to the next round.,Baker got through to the next round.,Baker was the Star Baker.,Baker was one of the judges' favourite bakers ...,Baker was one of the judges' least favourite b...,Baker was a series runner-up.,None,None,None,None,8
4,Stacey,Baker was one of the judges' least favourite b...,Baker was one of the judges' least favourite b...,Baker was one of the judges' favourite bakers ...,Baker got through to the next round.,Baker was one of the judges' favourite bakers ...,Baker was one of the judges' least favourite b...,Baker was one of the judges' least favourite b...,Baker was the Star Baker.,Baker was eliminated.,out,None,None,None,None,8
5,Liam,Baker got through to the next round.,Baker got through to the next round.,Baker was one of the judges' least favourite b...,Baker was one of the judges' favourite bakers ...,Baker was one of the judges' least favourite b...,Baker was the Star Baker.,Baker was one of the judges' favourite bakers ...,Baker was eliminated.,out,out,None,None,None,None,8
6,Yan,Baker got through to the next round.,Baker got through to the next round.,Baker got through to the next round.,Baker got through to the next round.,Baker got through to the next round.,Baker was one of the judges' favourite bakers ...,Baker was eliminated.,out,out,out,None,None,None,None,8
7,Julia,Baker got through to the next round.,Baker got through to the next round.,Baker was the Star Baker.,Baker got through to the next round.,Baker was one of the judges' least favourite b...,Baker was eliminated.,out,out,out,out,out,out,out,out,8
8,James,Baker got through to the next round.,Baker got through to the next round.,Baker got through to the next round.,Baker was one of the judges' least favourite b...,Baker was eliminated.,out,out,out,out,out,out,out,out,None,8
9,Tom,Baker got through to the next round.,Baker was one of the judges' favourite bakers ...,Baker got through to the next round.,Baker was eliminated.,out,out,out,out,out,out,out,out,out,None,8
10,Flo,Baker was one of the judges' favourite bakers ...,Baker was one of the judges' least favourite b...,Baker was eliminated.,out,out,out,out,out,out,out,out,out,None,None,8


In [449]:
import os

directory = 'season_{}'.format(season)

if not os.path.exists(directory):
    os.makedirs(directory)

season_df.to_csv('{}/episodes.csv'.format(directory))
bakers_df.to_csv('{}/bakers.csv'.format(directory))
elimination_df.to_csv('{}/elimination.csv'.format(directory))